# U-Net Depth (Projeto de dissertação)

## Clonar e navegar para o repositório

In [ ]:
import os

# Clone o repositório da dissertação
!git clone "https://github.com/duraes-antonio/unet_depth"

# Navegue para o repositório
!cd "unet_depth"
os.chdir("unet_depth")

## Definir variáveis de ambiente e credenciais para serviços

In [ ]:
import ast
import json

from kaggle_secrets import UserSecretsClient

running_remote = True
running_on_kaggle = True
db_name = 'unet_depth'

if running_remote:
    !pip install pymongo[srv] dnspython keras-unet-collection python-dotenv imutils
    !pip install py-cpuinfo gpuinfo

if running_on_kaggle:
    user_secrets = UserSecretsClient()
    database_url = user_secrets.get_secret("DATABASE_URL")
    google_cred_json = ast.literal_eval(user_secrets.get_secret("GOOGLE_CREDENTIALS_JSON"))
    google_token_json = ast.literal_eval(user_secrets.get_secret("GOOGLE_TOKEN_JSON"))

    with open('google_credentials.json', 'w', encoding='utf-8') as f:
        json.dump(google_cred_json, f, ensure_ascii=False, indent=4)

    with open('token.json', 'w', encoding='utf-8') as f:
        json.dump(google_token_json, f, ensure_ascii=False, indent=4)

    os.environ["DATABASE_URL"] = database_url
    os.environ["DB_NAME"] = db_name

## Instanciar serviços para persistência de resultados e blob

In [ ]:
from infra.services.blob_storage.blob_storage_service_google_drive import GoogleDriveBlobStorageService
from infra.services.model_storage_service_google_drive import ModelStorageServiceGoogleDrive
from domain.services.model_storage_service import ModelStorageService
from infra.services.test_case_execution_service_mongodb import TestCaseExecutionServiceMongoDB
from infra.services.test_case_service_mongodb import TestCaseServiceMongoDB
from domain.services.test_case_execution_service import TestCaseExecutionService
from domain.services.blob_storage_service import BlobStorageService
from domain.services.test_case_service import TestCaseService

test_case_serv: TestCaseService = TestCaseServiceMongoDB(db_name)
execution_serv: TestCaseExecutionService = TestCaseExecutionServiceMongoDB(db_name)

blob_service: BlobStorageService = GoogleDriveBlobStorageService()
model_storage: ModelStorageService = ModelStorageServiceGoogleDrive(blob_service)

### Buscar caso de teste para executar e modelo salvo previamente

In [ ]:
from domain.models.test_case import TestCaseState

last_test_case = test_case_serv.get_first_available()

# Se não tiver nenhum caso disponível, finalize a execução
if last_test_case is None:
    print('Todos casos de teste finalizados!')
    exit(0)
    raise InterruptedError()

test_case_serv.update_state(last_test_case['id'], TestCaseState.Busy)

# Buscar última execução do caso de teste
last_execution = execution_serv.get_last_execution(last_test_case['id'])

model_id = None

# Buscar último blob do modelo atualizado
if last_execution:
    model_id = last_execution['model_id']

    if model_id:
        model_storage.recover(model_id)

In [ ]:
from infra.util.output import print_test_case

print_test_case(last_test_case)

## Dataset

### Baixar dataset

In [ ]:
!git clone "https://gitlab.com/siddinc/new_depth.git" "./data"

### Particionar (em treino e teste) e instanciar geradores

In [ ]:
from domain.models.data.data_generator import NyuV2Generator
from infra.util.dataset import load_nyu_train_paths

train_path = "./data/nyu2_train.csv"
test_path = "./data/nyu2_test.csv"
batch_size = 4

# Defina a semente usada em operações pseudo-aleatórias (como embaralhamento do dataset)
seed = 42

partition, y_path_by_x_path = load_nyu_train_paths(train_path, 0.3, seed, dataset_usage_percent=0.1)

training_generator = NyuV2Generator(partition['train'], y_path_by_x_path, batch_size, seed=seed)
validation_generator = NyuV2Generator(partition['validation'], y_path_by_x_path, batch_size, seed=seed)

## Preparar modelo

### Definir configuração

In [ ]:
image_size = 256
input_shape = (image_size, image_size, 3)
width, height, n_channels = input_shape
epochs = 50
initial_lr = 0.0001

### Definir callbacks

In [ ]:
network = last_test_case['network'].value
optimizer: str = last_test_case['optimizer'].value
backbone = str(last_test_case['backbone'].value).lower()
use_image_net = int(last_test_case['use_imagenet_weights'])
epoch = last_execution['last_epoch'] + 1 if last_execution else 1

# Exemplo de formato: 'attention-unet_epoch-15_adam_resnet-101_imagenet-0'
trained_model_name = f'{network}_{optimizer}_{backbone}_imagenet-{use_image_net}'
csv_log_name = f'{trained_model_name}.csv'

In [ ]:
from infra.keras.metrics import build_poly_decay
from infra.keras.callbacks import CSVResultsSave, TrainedModelSaveRemote
from datetime import datetime
from tensorflow.python.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.python.keras.callbacks import TensorBoard, CSVLogger, ModelCheckpoint

tensorboard_log_path = "logs/fit/"
tensorboard_current_log_path = tensorboard_log_path + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

model_checkpoint_callback = ModelCheckpoint(
    filepath=trained_model_name,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)
save_model_remote_callback = TrainedModelSaveRemote(
    model_storage,
    execution_serv,
    test_case_serv,
    trained_model_name,
    last_test_case['id'],
    epoch
)

poly_decay = build_poly_decay(epochs, initial_lr)
callbacks = [
    LearningRateScheduler(poly_decay),
    EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True),
    CSVLogger(csv_log_name),
    CSVResultsSave(blob_service, csv_log_name, epoch),
    model_checkpoint_callback,
    save_model_remote_callback,
    TensorBoard(log_dir=tensorboard_current_log_path, histogram_freq=1),
]

## Compilar e treinar

In [ ]:
from infra.keras.model import build_model
from infra.keras.loss import depth_loss
from infra.keras.metrics import depth_acc
from tensorflow import keras

if model_id is not None and last_execution is not None:
    custom_objects = {'poly_decay': poly_decay, 'depth_loss': depth_loss, 'depth_acc': depth_acc}
    unet_model = keras.models.load_model(last_execution['model_name'], custom_objects)

else:
    unet_model = build_model(last_test_case, input_shape)

unet_model.compile(loss=depth_loss, metrics=[depth_acc], optimizer=optimizer)

In [ ]:
unet_model.fit(training_generator, validation_data=validation_generator, callbacks=callbacks, epochs=epochs)

In [ ]:
% load_ext tensorboard
% tensorboard --logdir $tensorboard_log_path

## Avaliar na base de teste

In [ ]:
from infra.util.dataset import read_nyu_csv

img_path_pairs = read_nyu_csv(test_path)
labels = {x_path: y_path for x_path, y_path in img_path_pairs}
labels_path = [y_path for x_path, y_path in img_path_pairs]

test_paths = [x_path for x_path, y_path in img_path_pairs]
partition = {'test': test_paths}

In [ ]:
from infra.util.preprocessing import preprocess_image
from infra.util.preprocessing import preprocess_depth_map
import numpy

x_test = numpy.empty((len(test_paths), height, width, n_channels))
y_test = numpy.empty((len(test_paths), height, width, 1))

for index, ID in enumerate(partition['test'][:]):
    x_test[index,] = preprocess_image(ID)
    y_test[index,] = preprocess_depth_map(labels[ID])

In [ ]:
unet_model.evaluate(x_test, y_test)

### Exibir comparação entre predição e ground truth

In [ ]:
from infra.util.output import plot_image_comparison

plot_image_comparison(unet_model, x_test, labels_path, 10)